In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy.stats as stats
import scipy.special
#graphing
import matplotlib.pyplot as plt
#stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

#import testing
import sys
sys.path.append("../")
import vuong_tests3

In [2]:
beta0 = 1.
beta1 = .25

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


yn,xn,nobs = gen_data()
print(xn.shape)
print(sm.add_constant(xn).shape)
print(scipy.stats.mode(yn))

(1000,)
(1000, 2)
ModeResult(mode=array([0.]), count=array([76]))


In [3]:
class Tobit(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(Tobit,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        m = 1*(self.endog == 0) #missingness
        
        beta = params
        
        mu_y = np.matmul(x,beta)
        
        pr_y = stats.norm.logpdf( y, loc = mu_y)
        
       
        #if complete case, assign pr missing to all observations...
        pr_m = stats.norm.logcdf( y, loc = mu_y)
        
        #we're done if ols
        if self.ols:
            return pr_y
        else:
            ll = (1-m)*pr_y + m*pr_m
            return ll
        
    def score(self, params):
        y = self.endog
        x = self.exog
        m = 1*(self.endog == 0) #missingness
        m_x = np.repeat(m,x.shape[1]).reshape(x.shape)
        
        if ols: #if OLS use all the data...
            m, m_x = np.ones(y.shape), np.ones(x.shape)
        
        
        b = params

        beta_jac = np.zeros(len(b))
        
        #for censored
        if not ols: 
            left_stats = (y - np.dot(x, b)) 
            l_pdf = scipy.stats.norm.logpdf(left_stats)
            l_cdf = scipy.stats.norm.logcdf(left_stats)
            left_frac = np.exp(l_pdf - l_cdf)
            beta_left = np.dot(left_frac*m, x*m_x)
            beta_jac -= beta_left
        
        #for non-censored
        mid_stats = (y - np.dot(x, b))
        beta_mid = np.dot(mid_stats*(1-m), x*(1-m_x) )
        beta_jac += beta_mid
        
        # by chain rule, since the expression above is dloglik/dlogsigma
        return beta_jac



model1 =  Tobit(yn,sm.add_constant(xn))
model1_fit = model1.fit(disp=False)
print(model1_fit.summary())

model2 =  Tobit(yn,sm.add_constant(xn),ols=True)
model2_fit = model2.fit(disp=False)
print(model2_fit.summary())

                                Tobit Results                                 
Dep. Variable:                      y   Log-Likelihood:                -1379.6
Model:                          Tobit   AIC:                             2763.
Method:            Maximum Likelihood   BIC:                             2773.
Date:                Tue, 23 Mar 2021                                         
Time:                        15:45:16                                         
No. Observations:                1000                                         
Df Residuals:                     998                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9939      0.047     21.200      0.000       0.902       1.086
x1             0.2508      0.007     37.151      0.0

In [4]:
def setup_shi(yn,xn):
    model1 = Tobit(yn,sm.add_constant(xn))
    model1_fit = model1.fit(disp=False)
    ll1 = model1.loglikeobs(model1_fit.params)
    grad1 =  model1.score_obs(model1_fit.params)    
    hess1 = model1.hessian(model1_fit.params)
    k1 = len(model1_fit.params)
    
    #fit logistic values
    model2 = Tobit(yn,sm.add_constant(xn),ols=True)
    model2_fit = model2.fit(disp=False)
    ll2 = model2.loglikeobs(model2_fit.params)
    grad2 =  model2.score_obs(model2_fit.params)    
    hess2 = model2.hessian(model2_fit.params)
    k2 = len(model2_fit.params)
    
    return ll1,grad1,hess1,ll2,k1, grad2,hess2,k2

yn,xn,nobs = gen_data()
ll1,grad1,hess1,ll2,k1, grad2,hess2,k2 = setup_shi(yn,xn)

In [21]:
beta0 = 1.
beta1 = .25

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


mc_out = vuong_tests3.monte_carlo(100,gen_data,setup_shi)
vuong_tests3.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 0.83 & 0.86 & 0.83 & 0.99   \\
Model 1 & 0.01 & 0.01 & 0.02 & 0.00   \\
Model 2 & 0.16 & 0.13 & 0.15 & 0.01   \\
\hline
\end{tabular}


In [33]:
beta0 = 1.
beta1 = .25

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 250
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs

mc_out = vuong_tests3.monte_carlo(100,gen_data,setup_shi)
vuong_tests3.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 0.88 & 0.87 & 0.99 & 0.98   \\
Model 1 & 0.00 & 0.02 & 0.00 & 0.00   \\
Model 2 & 0.12 & 0.11 & 0.01 & 0.02   \\
\hline
\end{tabular}


# Main examples

In [30]:
beta0 = 1.
beta1 = .25

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 500
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


mc_out = vuong_tests3.monte_carlo(100,gen_data,setup_shi)
vuong_tests3.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 0.87 & 0.89 & 0.97 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.13 & 0.11 & 0.03 & 0.00   \\
\hline
\end{tabular}


In [31]:
beta0 = 1.
beta1 = .5

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 500
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


mc_out = vuong_tests3.monte_carlo(100,gen_data,setup_shi)
vuong_tests3.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 0.00 & 0.00 & 0.00 & 0.10   \\
Model 1 & 1.00 & 1.00 & 1.00 & 0.90   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}


In [32]:
beta0 = 1.
beta1 = 1.

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 500
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


mc_out = vuong_tests3.monte_carlo(100,gen_data,setup_shi)
vuong_tests3.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 0.00 & 0.02 & 0.00 & 0.35   \\
Model 1 & 1.00 & 0.98 & 1.00 & 0.65   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}


# Misc examples

In [6]:
beta0 = 1.
beta1 = 2.

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


mc_out = vuong_tests3.monte_carlo(100,gen_data,setup_shi)
vuong_tests3.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 0.00 & 0.01 & 0.00 & 0.17   \\
Model 1 & 1.00 & 0.99 & 1.00 & 0.83   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}


In [26]:
def gen_data(beta0=beta0,beta1=1):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs

mc_out = vuong_tests3.monte_carlo(100,gen_data,setup_shi)
vuong_tests3.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 0.00 & 0.00 & 0.02 & 0.09   \\
Model 1 & 1.00 & 1.00 & 0.98 & 0.91   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}


In [27]:
def gen_data(beta0=beta0,beta1=1):
    nobs = 1000
    #parameters
    sigma = 1
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    m = beta0 + epsilon
    #censor
    y[m<=0] = 0
    return y,x,nobs
    
mc_out = vuong_tests3.monte_carlo(100,gen_data,setup_shi)
vuong_tests3.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 0.03 & 0.08 & 0.44 & 0.19   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.97 & 0.92 & 0.56 & 0.81   \\
\hline
\end{tabular}


In [28]:
beta0 = 1.
beta1 = .25

def gen_data(beta0=beta0,beta1=beta1):
    nobs = 1000
    #parameters
    sigma = 2
    
    epsilon = stats.norm.rvs(loc=0,scale=sigma,size=nobs)
    #censor data below x<0?
    x = stats.norm.rvs(loc=5,scale=5,size=nobs)
    y = beta0+ beta1*x + epsilon
    
    #censor
    y[y<=0] = 0
    return y,x,nobs


mc_out = vuong_tests3.monte_carlo(100,gen_data,setup_shi)
vuong_tests3.print_mc(mc_out)

\begin{tabular}{|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Shi (2015) \\ \hline \hline
No selection & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 1.00 & 1.00 & 1.00 & 1.00   \\
\hline
\end{tabular}
